In [1]:
import numpy as np

In [2]:
text = "The dog chased the cat"

In [3]:
dataPipes1 = np.zeros(shape=[5, 6])
dataPipes1

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

# self attention layer

## introduction

#### IF X = token embeddings / hidden states ของคำทั้งหมด

In [4]:
TheWord = np.array([1, 0, 1, 0, 0, 0])
dogWord = np.array([0, 0, 1, 0, 0, 0])
chasedWord = np.array([0, 1, 0, 0, 0, 0])
theWord = np.array([0, 1, 1, 0, 0, 0])
catWord = np.array([1, 0, 0, 0, 0, 0])

In [5]:
X = np.array([TheWord, dogWord, chasedWord, theWord, catWord])
print(X)

[[1 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 1 1 0 0 0]
 [1 0 0 0 0 0]]


In [6]:
dataPipes1 = X

#### K Q V ถูกสร้างจาก X ดังนี้
* $Q = X \cdot W_q$ - Q (query) : ฉันกำลังมองหาอะไร
* $K = X \cdot W_k$ - K (Key) : ฉันมีอะไรให้คนอื่นสนใจ
* $V = X \cdot W_v$ - V (value) : ข้อมูลจริงที่จะส่งต่อ

X คือคำ  
Q คือคำถาม  
K คือป้ายชื่อ  
V คือเนื้อหา

* $X ∈ R^{(n × d_{model})}$  
* $n$ = จำนวน tokens    
* $d_{model}$ = embedding size  
* $W_q ∈ R^{(d_{model} × d_k)}$  
* $W_k ∈ R^{(d_{model} × d_k)}$  
* $W_v ∈ R^{(d_{model} × d_v)}$  

In [7]:
n = 5 # 5 คำ ให้เป็น 5 โทเคน
d_model = 6 # example
num_heads = 2 # จำนวน attentions mattrix ที่ทำการคำนวณขนานกัน

In [8]:
#d_k = d_v = d_model / num_heads
d_k = int(d_model / num_heads)
d_v = int(d_model / num_heads)

In [9]:
W_Q1 = np.random.rand(d_model, d_k)
W_K1 = np.random.rand(d_model, d_k)
W_V1 = np.random.rand(d_model, d_v)

In [10]:
W_Q1

array([[0.04708922, 0.0642415 , 0.62869445],
       [0.62663345, 0.36858237, 0.10771409],
       [0.73704086, 0.11696681, 0.87654807],
       [0.38499175, 0.152832  , 0.45284694],
       [0.84038805, 0.12701202, 0.01372106],
       [0.53726775, 0.78251767, 0.65150939]])

In [11]:
W_Q2 = np.random.rand(d_model, d_k)
W_K2 = np.random.rand(d_model, d_k)
W_V2 = np.random.rand(d_model, d_v)

In [12]:
W_Q2

array([[0.00884881, 0.29564439, 0.38290652],
       [0.60639646, 0.72838854, 0.25969124],
       [0.95481531, 0.47983668, 0.31282952],
       [0.40932528, 0.575248  , 0.68495309],
       [0.87758026, 0.05904625, 0.69102643],
       [0.0296744 , 0.7114125 , 0.16633364]])

### attention #1

In [13]:
Q1 = np.dot(X, W_Q1)
K1 = np.dot(X, W_K1)
V1 = np.dot(X, W_V1)
Q1

array([[0.78413007, 0.18120831, 1.50524252],
       [0.73704086, 0.11696681, 0.87654807],
       [0.62663345, 0.36858237, 0.10771409],
       [1.36367431, 0.48554918, 0.98426216],
       [0.04708922, 0.0642415 , 0.62869445]])

### attention #2

In [14]:
Q2 = np.dot(X, W_Q2)
K2 = np.dot(X, W_K2)
V2 = np.dot(X, W_V2)
Q2

array([[0.96366411, 0.77548108, 0.69573603],
       [0.95481531, 0.47983668, 0.31282952],
       [0.60639646, 0.72838854, 0.25969124],
       [1.56121177, 1.20822523, 0.57252076],
       [0.00884881, 0.29564439, 0.38290652]])

### softmax
    

In [15]:
def softmax(x):
    x = x - np.max(x) # stability ป้องกันตัวเลข Overflow เป็นมาตรฐานใน deeplearning เพื่อป้องกันไม่ให้ค่าเฟ้อจากฟังก์ชัน exponential
    # print("current x is : ")
    # print(x)
    exp_x = np.exp(x)
    # print("current exp_x is : ")
    # print(exp_x)
    return exp_x / np.sum(exp_x)

## Attention Head 1

#### attention score of head 1

In [16]:
print(Q1.shape)
print(K1.shape)

(5, 3)
(5, 3)


$$score = \frac{Q \cdot K^T}{d_K} $$

In [17]:
scores1 = np.dot(Q1, K1.T) / np.sqrt(d_k)
scores1

array([[1.24990152, 0.53386101, 0.76671432, 1.30057533, 0.71604051],
       [0.92092261, 0.42552082, 0.51768351, 0.94320432, 0.49540179],
       [0.70151034, 0.40339074, 0.22278958, 0.62618032, 0.2981196 ],
       [1.62243295, 0.82891156, 0.74047309, 1.56938465, 0.79352139],
       [0.32897891, 0.1083402 , 0.24903082, 0.35737101, 0.22063871]])

$$scores ∈ R^{(n_{token} \times n_{token})}$$

#### apply softmax row by row 

In [18]:
attn1 = np.apply_along_axis(softmax, axis=1, arr=scores1)

In [19]:
attn1

array([[0.26710728, 0.13053088, 0.1647556 , 0.28099143, 0.15661481],
       [0.2529443 , 0.15412555, 0.16900532, 0.25864359, 0.16528124],
       [0.25269905, 0.18755641, 0.15656613, 0.23436254, 0.16881586],
       [0.30759023, 0.13910751, 0.12733337, 0.29169833, 0.13427056],
       [0.21499403, 0.17242658, 0.19847481, 0.22118564, 0.19291893]])

In [20]:
output1 = np.dot(attn1, V1)
output1

array([[0.37621345, 0.93931171, 0.76689662],
       [0.36638825, 0.91798948, 0.74944685],
       [0.35532414, 0.90396739, 0.74473661],
       [0.37972584, 0.96846659, 0.80908774],
       [0.3531473 , 0.87655188, 0.69888234]])

##### อธิบายความหมายของค่าที่ได้

In [21]:

#โฟกัสที่คำว่า "chased"

word_index = 2 # chased
print("attention weight: ", attn1[word_index])

attention weight:  [0.25269905 0.18755641 0.15656613 0.23436254 0.16881586]


ตรงนี้มี่ความหมายว่าคำว่า "chased" มีความสนใจคำอื่นมากแค่ไหน (อาจหมายถึงคำว่า "chased" มีความสัมพันธ์กับคำอื่นแค่ไหน)

In [22]:
print("output vector: ", output1[word_index])

output vector:  [0.35532414 0.90396739 0.74473661]


เป็นเวคเตอร์ใหม่ของคำว่า "chased" ที่มีบริบทรวมอยู่เรียบร้อยแล้ว

## Attention head 2

In [23]:
scores2 = np.dot(Q2, K2.T)/np.sqrt(d_k)
print("scores from second attendtion head 2 ->")
print(scores2)

attn2 = np.apply_along_axis(softmax, axis=1, arr=scores2);
print("attention2 =>")
print(attn2)

output2 = np.dot(attn2, V2)
print("output 2 =>");
print(output2)

scores from second attendtion head 2 ->
[[2.00874993 1.26376676 0.96099434 2.2247611  0.74498318]
 [1.45654521 0.89596274 0.64621955 1.54218229 0.56058248]
 [1.24733046 0.83894039 0.67471074 1.51365113 0.40839007]
 [2.70387567 1.73490313 1.32093029 3.05583342 0.96897254]
 [0.55220472 0.36780402 0.31477479 0.68257881 0.1844007 ]]
attention2 =>
[[0.29858143 0.14174923 0.10471968 0.37057426 0.0843754 ]
 [0.28463957 0.16249403 0.12658298 0.31008945 0.11619397]
 [0.25213768 0.16760084 0.14221721 0.32907838 0.10896589]
 [0.30973397 0.11753559 0.07769325 0.44039476 0.05464243]
 [0.22463016 0.186803   0.17715506 0.25591092 0.15550086]]
output 2 =>
[[0.77208111 1.17210328 1.1999874 ]
 [0.74637438 1.11250824 1.13447718]
 [0.74552262 1.10160012 1.14100605]
 [0.80113521 1.23621345 1.27118059]
 [0.71769633 1.0169138  1.05623706]]


## Now we got the 2 output from both attention head 1 and 2

#### let concat it together

In [24]:
output = np.concatenate([output1, output2], axis=1)
output

array([[0.37621345, 0.93931171, 0.76689662, 0.77208111, 1.17210328,
        1.1999874 ],
       [0.36638825, 0.91798948, 0.74944685, 0.74637438, 1.11250824,
        1.13447718],
       [0.35532414, 0.90396739, 0.74473661, 0.74552262, 1.10160012,
        1.14100605],
       [0.37972584, 0.96846659, 0.80908774, 0.80113521, 1.23621345,
        1.27118059],
       [0.3531473 , 0.87655188, 0.69888234, 0.71769633, 1.0169138 ,
        1.05623706]])

In [25]:
output.shape

(5, 6)

In [26]:
X.shape

(5, 6)

In [27]:
dataPipes1 = output

now output from attention layer have the same dimension and size as X

## Output projection after concat output

**Output Projection** คือ **Linear transformation** สุดท้ายหลังจากรวม (concat) ผลลัพธ์ของทุก attention heads 
เพื่อผสมมุมมองทั้งหมด และทำให้ขนาดกลับไปเป็น $d_{model}$

สัญลักษณ์ใน Paper คือ $W_O$

ดำเนินการหลังจาก concat เป็น output จาก multi-head attention เรียบร้อยแล้วโดยการ

$$Output_{final} = Concat(output_1, output_2, ... , output_n) \cdot W_O$$

* ส่วนนี้เกิดขึ้นมาจากการที่แต่ละ Head ทำงานแยกขาดจากกัน
* ทำให้ไม่มีการผสมข้อมูลระหว่างแต่ละ attention head 
* ทำให้โมเดลเสียความสามารถในการเรียนรู้

**Output Projection** ทำหน้าที่
* ผสมข้อมูลจากทุก Head 
* เรียนรู้ว่าควรเน้นข้อมูลจาก Head ไหน
* จัด alignment กลับเข้าสู่ space หลัก ($d_{model}$)

In [28]:
#example W_O

W_O = np.random.rand(d_model, d_model)
W_O

array([[0.68784748, 0.42098354, 0.76932261, 0.74871514, 0.69315673,
        0.69587133],
       [0.22051393, 0.27399305, 0.99130393, 0.61347141, 0.68167614,
        0.60199781],
       [0.54129466, 0.1802292 , 0.67077934, 0.43062251, 0.07167981,
        0.44036641],
       [0.356506  , 0.10744546, 0.66250021, 0.03377392, 0.88027367,
        0.85006274],
       [0.56645404, 0.70040163, 0.31374393, 0.77667224, 0.13474257,
        0.05153243],
       [0.79349427, 0.91238452, 0.46028566, 0.20869622, 0.99592442,
        0.4991076 ]])

In [29]:
attention_output = np.dot(output, W_O)
attention_output

array([[2.77240315, 2.55271131, 3.16657248, 2.37500966, 2.9887239 ,
        2.48061679],
       [2.65659315, 2.43531491, 3.06029027, 2.28623278, 2.87022567,
        2.39563943],
       [2.64203906, 2.42419149, 3.03373722, 2.26018018, 2.85694305,
        2.37939717],
       [2.90725084, 2.68276244, 3.29860866, 2.47932519, 3.11917394,
        2.58272632],
       [2.48452069, 2.26785298, 2.89010463, 2.13758154, 2.71312972,
        2.27085858]])

**from paper :**  
Output projection lets the model recombine information from different attention heads into a unified representation.

## Add & Norm step after get final output from multi-head attention

ใน 1 Transformer block จะมีการ Add & Norm 2 ครั้ง

ครั้งแรกเกิดหลังจากการทำ Multi-head Attention
ครั้งที่สองเกิดหลังการทำ Feed-Forward Network

โดยการนำ Embedding Vector หรือ X มาบวกเข้ากับ Final output จาก multi-head attention แล้วทำการ Normalized เพื่อให้ค่าใน Vector ไม่โตจนเกินไป

### การ ADD

$$Y = X + F(X)$$
โดย 
* $X$ คือ input เดิม
* $F(X)$ คือ output จาก multi-head attention or FFN

กระบวนการนี้เกิดจากการที่
* เมื่อค่าที่ได้จาก transformer block ผ่านไปยังอีก block ที่อยู่ลึกมาก ๆ แล้ว gradient หายไประหว่างทาง (Vanishing gradient)
* ทำให้การ training ทำได้ยาก
* และข้อมูลเดิมหายระหว่างทาง

โมเดลควรเรียนรู้ว่า "ฉันควรเพิ่มอะไรจากของเดิม"

In [30]:
residual1 = X + dataPipes1
residual1

array([[1.37621345, 0.93931171, 1.76689662, 0.77208111, 1.17210328,
        1.1999874 ],
       [0.36638825, 0.91798948, 1.74944685, 0.74637438, 1.11250824,
        1.13447718],
       [0.35532414, 1.90396739, 0.74473661, 0.74552262, 1.10160012,
        1.14100605],
       [0.37972584, 1.96846659, 1.80908774, 0.80113521, 1.23621345,
        1.27118059],
       [1.3531473 , 0.87655188, 0.69888234, 0.71769633, 1.0169138 ,
        1.05623706]])

In [31]:
X

array([[1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [32]:
residual1

array([[1.37621345, 0.93931171, 1.76689662, 0.77208111, 1.17210328,
        1.1999874 ],
       [0.36638825, 0.91798948, 1.74944685, 0.74637438, 1.11250824,
        1.13447718],
       [0.35532414, 1.90396739, 0.74473661, 0.74552262, 1.10160012,
        1.14100605],
       [0.37972584, 1.96846659, 1.80908774, 0.80113521, 1.23621345,
        1.27118059],
       [1.3531473 , 0.87655188, 0.69888234, 0.71769633, 1.0169138 ,
        1.05623706]])

### Norm (Layer Normalization) 


ทำให้ค่าใน vector 
* $mean \approx 0$
* $variance \approx 1$

$$LayerNorm(x) = \frac{x - \mu}{\sigma} \cdot \gamma + \beta$$

ในกรณีที่ไม่มีการ Normalized 
* ทำให้ Vector โตขึ้นเรื่อย ๆ
* scale เพี้ยนเรื่อย ๆ 
* training ไม่นิ่ง

In [33]:
def layer_norm (x, eps=1e-6):
    mean = np.mean(x, axis=1, keepdims=True)
    print("mean = ", mean)
    std = np.std(x, axis=1, keepdims=True)
    print("std = ", std)
    return (x - mean) / (std + eps)

In [34]:
norm1 = layer_norm(residual1)

mean =  [[1.20443226]
 [1.00453073]
 [0.99869282]
 [1.24430157]
 [0.95323812]]
std =  [[0.31732115]
 [0.42126895]
 [0.48181552]
 [0.54590505]
 [0.22384828]]


In [35]:
norm1

array([[ 0.54134634, -0.83549338,  1.77253419, -1.36249911, -0.10188063,
        -0.0140074 ],
       [-1.51480655, -0.20542944,  1.76826314, -0.61280505,  0.25631431,
         0.3084636 ],
       [-1.3352981 ,  1.87887821, -0.52708074, -0.52544941,  0.21358193,
         0.2953681 ],
       [-1.58374456,  1.32653783,  1.03458493, -0.81179969, -0.01481595,
         0.04923745],
       [ 1.78651094, -0.3425798 , -1.13628147, -1.05223385,  0.28445785,
         0.46012633]])

### eps value

eps = $\epsilon$ 
* เป็นค่าคงที่เล็ก ๆ ใช้ป้องกันการหารด้วย 0
* มักจะมีค่าเท่ากับ $1 \times 10^{-6}$ เนื่องจากเป็นค่าที่เล็กพอที่จะไม่รบกวนผลลัพธ์แต่ใหญ่พอที่จะป้องกัน numerical error

### $\beta$ and $\gamma$ หายไปไหน

$\beta$ และ $\gamma$
เป็นค่าที่ได้จากการ training 
ทำให้โมเดลสามารถเลือกได้ว่า 
* dimension ไหนควรเพิ่ม - ลด และ 
* dimension ไหนควรเลื่อนค่า

In [36]:
#for example beta and gamma after training method is 
beta1 = np.array([2, 0.5, 2, 1, 1, 1])
gamma1 = np.array([0.1, -0.2, 0.3, -0.12, 0.32, 0.2])

In [37]:
y = gamma1 * norm1 + beta1
y

array([[2.05413463, 0.66709868, 2.53176026, 1.16349989, 0.9673982 ,
        0.99719852],
       [1.84851934, 0.54108589, 2.53047894, 1.07353661, 1.08202058,
        1.06169272],
       [1.86647019, 0.12422436, 1.84187578, 1.06305393, 1.06834622,
        1.05907362],
       [1.84162554, 0.23469243, 2.31037548, 1.09741596, 0.99525889,
        1.00984749],
       [2.17865109, 0.56851596, 1.65911556, 1.12626806, 1.09102651,
        1.09202527]])

In [38]:
dataPipes1 = norm1

# Feed Forward neural network

สิ่งที่ได้จากกระบวนการนี้
$$H \in R^{(n \times d_{model})}$$

* FFN ทำงานกับ H ทีละ token (row-wise)
* FFN จะไม่ดู token อื่นเลย (ไม่ขึ้นกับ token อื่น)

FFN definition from Attention is All you need: 
    $$FFN(x) = max(0, x \cdot W_1 + b_1)W_2 + b_2$$

(ของใหม่จะใช้ GELU แทน ReLU)

## 1st linear (expand)

expand dimension from 6 to 24 (4 times of $n_{model}$)

In [39]:
# this value came from training method
W_FFN_Expand_1 = np.random.rand(6, 24) 
W_FFN_Shrink_1 = np.random.rand(24, 6)

In [40]:
# this value came from training method
b_FFN_Expand_1 = np.random.rand(1, 24)
b_FFN_Shrink_2 = np.random.rand(1, 6)

#### *add by using broadCasting*
 * $dataPipes_1 \in R^{(n_{model} \times 5)}$ : result from Plus & Norm process have same dimensions of X 
 * $W_1 \in R^{(n_{model} \times 4n_{model})}$ : layer แรกเป็นเลเยอร์ขยายผลลัพธ์ที่ได้จากเลเยอร์นี้มักจะมีขนาดมิติเป็น 4 เท่าของค่า $n_{model}$

In [41]:
h1 = np.dot(dataPipes1, W_FFN_Expand_1) + b_FFN_Expand_1
h1.shape

(5, 24)

### pass to Relu function

In [42]:
h_prime = np.maximum(0, h1) # if child of h1 less than 0 return 0 instead of minus value inside h1
h_prime

array([[0.06479924, 0.70402513, 1.71824605, 0.        , 0.88001057,
        0.10353469, 1.45341774, 0.14105698, 1.3688423 , 0.60295556,
        0.        , 0.26646677, 0.09999023, 0.        , 2.13756204,
        1.27889581, 0.29089771, 0.66184581, 0.        , 1.71283551,
        1.23783741, 0.        , 1.5195032 , 1.1610103 ],
       [0.69446813, 0.15953848, 0.65948524, 0.        , 0.87910651,
        1.12408548, 1.09500518, 0.        , 1.76739853, 0.        ,
        0.        , 0.47315002, 1.30590136, 0.        , 1.72645648,
        1.81132986, 0.80244623, 1.49718033, 0.        , 1.83040407,
        1.15304348, 0.        , 1.19722163, 0.16674916],
       [0.38697682, 1.23530537, 0.04304708, 0.        , 0.19088835,
        1.69825204, 0.46882305, 0.47701093, 0.83948631, 0.        ,
        0.        , 0.        , 2.41999214, 0.46004785, 0.        ,
        0.13370744, 1.56783938, 1.28969916, 0.        , 0.16666604,
        0.23767671, 0.88069751, 0.16034111, 0.33221245],
       [0.784

## 2st linear (Shrink)

then dot with shrink weight and shrink bias again

In [43]:
output = np.dot(h_prime, W_FFN_Shrink_1) + b_FFN_Shrink_2
output.shape

(5, 6)

In [44]:
print(output)

[[7.74917104 6.77829183 7.42881085 9.23922132 8.69611043 7.27749788]
 [8.11571798 6.13037045 7.93791937 9.20765093 9.44132199 7.27573054]
 [5.25498793 4.35408296 5.48604111 6.58328037 6.21880004 6.80875205]
 [6.10892574 4.7195013  6.51063603 7.18971822 7.60325369 6.85262231]
 [5.57559468 5.76495452 5.11863298 7.12567923 6.02234862 7.12151798]]


## conclude

In [45]:
def ffn(X, W1, b1, W2, b2):
    h = np.dot(X, W1) + b1
    h = np.maximum(0, h)
    h = np.dot(h, W2) + b2
    return h

In [46]:
dataPipes1 = ffn(dataPipes1, W_FFN_Expand_1, b_FFN_Expand_1, W_FFN_Shrink_1, b_FFN_Shrink_2)
dataPipes1.shape

(5, 6)

# Add and Norm once again at the end of transformer block

In [47]:
addResult = dataPipes1 + X
norm2 = layer_norm(addResult)
norm2.shape

mean =  [[8.19485056]
 [8.18478521]
 [5.95099074]
 [6.83077622]
 [6.28812133]]
std =  [[0.87102956]
 [1.17218604]
 [0.61429681]
 [0.70050547]
 [0.72923885]]


(5, 6)

In [48]:
dataPipes1 = norm2
dataPipes1

array([[ 0.63639613, -1.62630199,  0.26860171,  1.19900588,  0.57547909,
        -1.05318082],
       [-0.05892168, -1.75263391,  0.6425034 ,  0.87261306,  1.07195928,
        -0.77552015],
       [-1.13300552, -0.97169122, -0.75687984,  1.02928842,  0.43596003,
         1.39632812],
       [-1.03046939, -1.5863878 ,  0.9705261 ,  0.51240355,  1.10274138,
         0.03118615],
       [ 0.3942096 , -0.71741392, -1.60370879,  1.14853556, -0.36445172,
         1.14282927]])

# Let's move to final step before we get the next token

after the data pipes pass through transformer block n block (currently n = 1 lol) we got

In [51]:
print(dataPipes1)

[[ 0.63639613 -1.62630199  0.26860171  1.19900588  0.57547909 -1.05318082]
 [-0.05892168 -1.75263391  0.6425034   0.87261306  1.07195928 -0.77552015]
 [-1.13300552 -0.97169122 -0.75687984  1.02928842  0.43596003  1.39632812]
 [-1.03046939 -1.5863878   0.9705261   0.51240355  1.10274138  0.03118615]
 [ 0.3942096  -0.71741392 -1.60370879  1.14853556 -0.36445172  1.14282927]]


In [52]:
dataPipes1.shape

(5, 6)

ในการทำนาย Token ถัดไป 

เราจะใช้ hidden state ของตำแหน่งสุดท้ายเท่านั้น

In [54]:
last_hidden = dataPipes1[-1]
print(last_hidden)
print('shape of lass hidden state : ', last_hidden.shape)

[ 0.3942096  -0.71741392 -1.60370879  1.14853556 -0.36445172  1.14282927]
shape of lass hidden state :  (6,)


ในเชิงความหมายในขั้นตอนนี้

เวคเตอร์นี้คือ "โมเดลเข้าใจประโยคทั้งหมดแล้ว และพร้อมจะเดาคำถัดไป"

Shape : 
$$last\ hidden \in R^{d_{model}}$$ หรือ $$R^{(d_{model}, )}$$

เป้าหมายขั้นสุดท้าย

สมมุติให้ vocab ทั้งหมดมี 20 ตัว
จะได้ output เป็น

$$P(token_1), P(token_2), ..., P(token_{19})$$

ดังนั้นเราต้องแปลง: array จากขนาด $6 \rightarrow 20$

ซึ่ง Transformer block ทำไม่ได้
เป็นหน้าที่ของ LM Head (Output Linear Layer)

## Last Linear layer (LM Head)

Mathemetic definition

$$Logits = h_{last} \cdot W_{vocab} + b_{vocab}$$

Where 
* $h_{last} \in R^{(d_{model}, )}$
* $W_{vocab} \in R^{(d_{model} \times vocab\_size)}$
* $b_{vocab} \in R^{(vocab\_size,)}$ 
* $logits \in R^{(vocab\_size,)}$ 

In [55]:
print("d_model =", d_model)

d_model = 6


In [56]:
vocab_size = 20

W_vocab = np.random.randn(d_model, vocab_size)
b_vocab = np.random.randn(vocab_size)

In [58]:
print(W_vocab.shape)

(6, 20)


In [59]:
logits = np.dot(last_hidden, W_vocab) + b_vocab

In [60]:
print(logits)

[ 3.05492994  3.04876717  0.10503566 -0.08815435  2.14139093 -2.3520933
 -3.40995254  4.67951357 -3.81675782 -0.67168809  1.88311049  0.35795136
 -2.11323825 -1.29417076  1.21322951 -0.75312505 -0.32719262 -1.49954131
 -0.33330187 -0.06028637]


if the logit result is   
[ 3.05492994  3.04876717  0.10503566 -0.08815435  2.14139093 -2.3520933
 -3.40995254  4.67951357 -3.81675782 -0.67168809  1.88311049  0.35795136
 -2.11323825 -1.29417076  1.21322951 -0.75312505 -0.32719262 -1.49954131
 -0.33330187 -0.06028637]

ความหมายคือ:
* token #8 (logit = 4.67) มีโอกาสมากกว่า token #1 (logit = 3.05)
* ค่ายังไม่ใช่ probability
* เป็นแค่คะแนนความเหมาะสม

### Softmax -> probability

Softmax ทำแค่ "normalize"

In [61]:
probs = softmax(logits)
probs.shape

(20,)

In [62]:
probs

array([1.20523928e-01, 1.19783451e-01, 6.30885388e-03, 5.20054809e-03,
       4.83424134e-02, 5.40546328e-04, 1.87676757e-04, 6.11816214e-01,
       1.24950271e-04, 2.90150705e-03, 3.73386453e-02, 8.12438252e-03,
       6.86382737e-04, 1.55697931e-03, 1.91087793e-02, 2.67458256e-03,
       4.09483226e-03, 1.26791911e-03, 4.06989213e-03, 5.34751515e-03])

### ความหมายเชิงผลลัพธ์

In [64]:
for i, p in enumerate(probs):
    print(f"token_{i}: {p:.4f}")

token_0: 0.1205
token_1: 0.1198
token_2: 0.0063
token_3: 0.0052
token_4: 0.0483
token_5: 0.0005
token_6: 0.0002
token_7: 0.6118
token_8: 0.0001
token_9: 0.0029
token_10: 0.0373
token_11: 0.0081
token_12: 0.0007
token_13: 0.0016
token_14: 0.0191
token_15: 0.0027
token_16: 0.0041
token_17: 0.0013
token_18: 0.0041
token_19: 0.0053


In [66]:
next_token_id = np.argmax(probs)
next_token_id

np.int64(7)

In [67]:
next_token_id = np.random.choice(
    range(vocab_size),
    p=probs
)

In [68]:
next_token_id

np.int64(7)